In [1]:
%matplotlib inline


# Computing Energy and Force Using Models Inside Model Zoo

TorchANI has a model zoo trained by NeuroChem. These models are shipped with
TorchANI and can be used directly.


To begin with, let's first import the modules we will use:



In [2]:
import torch
import torchani

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


Let's now manually specify the device we want TorchANI to run:



In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Let's now load the built-in ANI-1ccx models. The builtin ANI-1ccx contains 8
models trained with diffrent initialization. Predicting the energy and force
using the average of the 8 models outperform using a single model, so it is
always recommended to use an ensemble, unless the speed of computation is an
issue in your application.

The ``periodic_table_index`` arguments tells TorchANI to use element index
in periodic table to index species. If not specified, you need to use
0, 1, 2, 3, ... to index species



In [4]:
model = torchani.models.ANI2x(periodic_table_index=True).to(device)

/Users/sergiortizropero/miniconda3/envs/ASE_ANI/lib/python3.10/site-packages/torchani/resources/


Now let's define the coordinate and species. If you just want to compute the
energy and force for a single structure like in this example, you need to
make the coordinate tensor has shape ``(1, Na, 3)`` and species has shape
``(1, Na)``, where ``Na`` is the number of atoms in the molecule, the
preceding ``1`` in the shape is here to support batch processing like in
training. If you have ``N`` different structures to compute, then make it
``N``.

<div class="alert alert-info"><h4>Note</h4><p>The coordinates are in Angstrom, and the energies you get are in Hartree</p></div>



In [5]:
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

Now let's compute energy and force:



In [6]:
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative

And print to see the result:



In [7]:
print('Energy:', energy.item())
print('Force:', force.squeeze())

Energy: -40.45979070722928
Force: tensor([[ 0.0478, -0.1304, -0.0551],
        [-0.1353,  0.1581, -0.0776],
        [ 0.0804, -0.0388,  0.0387],
        [ 0.0254,  0.0076,  0.0433],
        [-0.0183,  0.0035,  0.0508]])


you can also get the atomic energies (WARNING: these have no physical
meaning) by calling:



In [8]:
_, atomic_energies = model.atomic_energies((species, coordinates))

this gives you the average (shifted) energies over all models of the ensemble by default,
with the same shape as the coordinates. Dummy atoms, if present, will have an
energy of zero



In [9]:
print('Average Atomic energies, for species 6 1 1 1 1', atomic_energies)

Average Atomic energies, for species 6 1 1 1 1 tensor([[-38.0796,  -0.5806,  -0.5908,  -0.6047,  -0.6040]],
       grad_fn=<MeanBackward1>)


you can also access model specific atomic energies



In [10]:
_, atomic_energies = model.atomic_energies((species, coordinates), average=False)
print('Atomic energies of first model, for species 6 1 1 1 1', atomic_energies[0, :, :])

Atomic energies of first model, for species 6 1 1 1 1 tensor([[-38.0841,  -0.5797,  -0.5898,  -0.6034,  -0.6027]],
       grad_fn=<SliceBackward0>)
